In [7]:

!pip install sqlalchemy

     ---------------------------------------- 1.6/1.6 MB 7.6 MB/s eta 0:00:00
     -------------------------------------- 101.9/101.9 kB 6.1 MB/s eta 0:00:00


In [28]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [32]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [33]:
sql = "SELECT * FROM {}".format("교통량_2021")
df = pd.read_sql(sql, db_connection)

In [34]:
sql2 = "select * from {}".format("MainData")
df2 = pd.read_sql(sql2, db_connection)

In [35]:
df2

,번호,위도,경도,생태통로_유형,차선_수,등산객_이용빈도,생태통로_효율성,이용확인종,이용확인종_수,폭(m),...,식물군락명,등산로까지 최단거리(km),유도울타리_위도_1km,유도울타리_경도_1km,유도울타리_위도_2.3km,유도울타리_경도_2.3km,유도울타리_위도_5km,유도울타리_경도_5km,유도울타리_연장_m,유도울타리_높이_m
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,...,소나무군락,3.157026,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,...,소나무-신갈나무군락,1.087578,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,소나무군락,3.070682,0.0,0.0,0.000000,0.000000,36.927754,128.333896,1028,2.000000
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,...,소나무군락,2.921190,0.0,0.0,0.000000,0.000000,36.927754,128.333896,1028,2.000000
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,8.23,...,아까시나무-리기다소나무식재림,0.025083,0.0,0.0,36.708802,127.437539,36.708802,127.437539,886,1.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,13.30,...,아까시나무-리기다소나무식재림,0.025617,0.0,0.0,36.701443,127.431078,36.701443,127.431078,442,1.000000
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,13.60,...,아까시나무식재림,0.021407,0.0,0.0,36.708802,127.437539,36.708802,127.437539,886,1.571429
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,13.30,...,아까시나무-리기다소나무식재림,0.008204,0.0,0.0,36.708802,127.437539,36.708802,127.437539,886,1.571429
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,7.00,...,상수리나무군락,0.878327,0.0,0.0,36.759335,127.483618,36.759335,127.483618,884,1.500000


In [7]:
!pip install haversine

In [36]:
from haversine import haversine
import numpy as np

In [37]:
(df2[:]["위도"][1])

37.03408

In [38]:
df.columns

Index(['연도', '도로종류', '지점번호', '호선', '구간', '차로', '도/시', '시/군', '읍/면', '동/리',
       '구분', 'AADT', 'XCODE', 'YCODE'],
      dtype='object')

In [40]:
count = 0
index = []
for i in range(len(df2)):
    for j in range(len(df)):
        dist = haversine((df2[:]["위도"][i],df2[:]["경도"][i]), (df[:]["XCODE"][j],df[:]["YCODE"][j]), unit = 'km')
        count +=1
        if dist <= 7.82 : #각 구간간 거리평균이 7.82
            index.append([i,j])
            

In [41]:
index2 = pd.DataFrame(index)
index2

,0,1
0,0,1180
1,0,1182
2,0,1183
3,0,1649
4,0,1651
...,...,...
4328,496,1743
4329,496,2535
4330,496,2538
4331,496,2590


In [42]:
a=pd.DataFrame(df[["차로", "AADT"]].iloc[index2[1]])
a = a.reset_index()
#index3.head(10)

In [43]:
index3 = pd.concat([index2, a],axis =1)
final = index3[[0,"AADT"]].groupby(0).mean()
final

,AADT
0,
0,7493.875000
1,3573.857143
2,3430.428571
3,3430.428571
4,34542.400000
...,...
492,34270.190476
493,34542.400000
494,34270.190476


In [47]:
final2 = pd.concat([df2, final], axis = 1)
final3 = final2[['번호', "AADT"]]
final3.count()
#결측값 약 200개 -> 평균으로 대체함
final3 = final3.fillna(final3.mean())
final3.count()

번호      497
AADT    497
dtype: int64

In [49]:
#데이터 디비에 올리기
final3.to_sql(name = "MainData_도은", con=db_connection, if_exists='replace', index=False)